In [1]:
import cifar10
from ops import *
import time

In [2]:
class TripleGan(object):
    
    def discriminator(self,x,label,scope='discriminator',is_training=True,reuse=False):
        with tf.variable_scope(scope, reuse=reuse):
            x=dropout(x,0.2)  #to prevent overfitting, probability is kept at 0.2
            y=reshape(label,[-1,1,1,10])#reshaping label to a 4-D vector
            x=conv_concat(x,y)#concatenating y to x by reshaping y to same dimensions as x and adding ones wherever necessary 
        
        #convolution 1
        x=conv_layer(x,filter_size=32,stride=1,kernel=[3,3])#convolution with stride=1 
        x=lrelu(x,0.2)#adding rectified linear unit to prevent vanishing derivatives
        x=conv_concat(x,y)#concatenating y for conditional discriminator so that we can condition it on basis of output as well 32*128
        x=conv_layer(x,filter_size=32,stride=2,kernel=[3,3])#strides of 2 as x has now doubled because of conactenating y to it 32*64
        x=dropout(x,0.2)
        x=lrelu(x,0.2)
        
        
        #convolution 2
        x=conv_layer(x,filter_size=64,stride=1,kernel=[3,3])
        x=lrelu(x,0.2)
        x=conv_concat(x,y)
        x=conv_layer(x,filter_size=64,stride=2,kernel=[3,3])
        x=dropout(x,0.2)
        x=lrelu(x,0.2) 
        
        
        #convolution 3
        x=conv_layer(x,filter_size=64,stride=1,kernel=[3,3])
        x=lrelu(x,0.2)
        x=conv_concat(x,y)
        x=conv_layer(x,filter_size=64,stride=2,kernel=[3,3])
        x=lrelu(x,0.2) 
        
        #FC layers
        x=GAP(x)#global average pooling layer, used to prevent overfitting reduces dimensions of form h*w*d to form 1*1*d by taking averages of h and w values
        x=flatten(x)#flattening to a 1-d array
        x=concat(x,label)#concatenation of labels to the output after all convolution to the flattened layer,1st fully conneceted layer
        logit=linear(x,unit=1)
        output=sigmoid(logit)#activation function
        
        return output,logit,x
        